In [1]:
%load_ext pycodestyle_magic
%load_ext mypy_ipython
%pycodestyle_on

In [2]:
import doctest

In [3]:
class Integer:

    def __init__(self, name):
        self.name = name

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError(f'expected int but got {type(value).__name__}')

        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]


class Point:
    x = Integer('x')
    y = Integer('y')

    def __init__(self, x, y):
        self.x = x
        self.y = y


"""

>>> p = Point(2, 3)
>>> p.x
2
>>> p.y = 5
>>> p.x = 1.23
Traceback (most recent call last):
    ...
TypeError: expected int but got float
"""

doctest.testmod()

TestResults(failed=0, attempted=4)

In [4]:
class Typed:

    def __init__(self, name, type_expected):
        self.name = name
        self.type_expected = type_expected

    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if not isinstance(value, self.type_expected):
            raise TypeError(f'expected {self.type_expected.__name__}'
                            f' but got {type(value).__name__}')

        instance.__dict__[self.name] = value

    def __delete__(self, instance):
        del instance.__dict__[self.name]


def typeassert(**kwargs):
    def decorator(cls):
        for name, expected_type in kwargs.items():
            setattr(cls, name, Typed(name, expected_type))
        return cls

    return decorator


@typeassert(name=str, shares=int, price=float)
class Stock:

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


"""

>>> s = Stock('pencil', 3, 0.5)
>>> s.name = 0
Traceback (most recent call last):
    ...
TypeError: expected str but got int
>>> s.shares = '1'
Traceback (most recent call last):
    ...
TypeError: expected int but got str
>>> s.price = 'pumpkin'
Traceback (most recent call last):
    ...
TypeError: expected float but got str
"""

doctest.testmod()

TestResults(failed=0, attempted=4)